In [ ]:
!pip install PyMuPDF
import fitz  # PyMuPDF
import re
import csv

def get_texts_matching_cases(pdf_path, case_list, output_csv, start_page=0, end_page=None):
    doc = fitz.open(pdf_path)
    matching_texts = []
    # 如果沒有指定end_page，則處理到PDF的最後一頁
    if end_page is None:
        end_page = len(doc)

    for page_num in range(start_page, min(end_page, len(doc))):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        # ...（其餘代碼與之前相同）...

    # 處理完後不關閉doc，允許下次調用時繼續處理
    # doc.close()

    # 在這裡可以選擇是追加到CSV或者是先保存到暫時文件
    # 以追加模式打開CSV文件
    with open(output_csv, 'a', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # 如果是文件的開始部分，則寫入表頭
        if start_page == 0:
            writer.writeheader()
        # 寫入數據
        for result in matching_texts:
            writer.writerow(result)

    # 返回最後處理的頁面號，以便進行下一段處理
    return page_num

# 示範用法
pdf_file_path = '/content/drive/MyDrive/textmining/Files (100).PDF'
output_csv = '/content/drive/MyDrive/textmining/100.csv'
result_list = []  # Create an empty list to store case names

# 定義每次處理的頁面數
pages_at_a_time = 50  # 或根據您的機器性能選擇一個合適的數字

# 初始呼叫
last_page_processed = get_texts_matching_cases(pdf_file_path, result_list, output_csv, 0, pages_at_a_time)

# 循環處理剩餘的頁面
while last_page_processed < len(fitz.open(pdf_file_path)):
    last_page_processed = get_texts_matching_cases(pdf_file_path, result_list, output_csv, last_page_processed + 1, last_page_processed + 1 + pages_at_a_time)



In [ ]:
import fitz  # PyMuPDF

def find_core_terms_pages(pdf_path, search_term):
    doc = fitz.open(pdf_path)
    pages_with_term = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text_instances = page.search_for(search_term)

        # 如果在當前頁面找到了該字樣，將頁碼加入列表
        if text_instances:
            pages_with_term.append(str(page_num + 1))  # 加1是因為頁碼通常從1開始

    doc.close()

    # 將頁碼列表轉換為逗號分隔的字串
    return ','.join(pages_with_term)

# 示範用法
pdf_file_path = '/content/drive/MyDrive/textmining/Files (100).PDF'
search_term = "Core Terms"

# 呼叫函數並列印結果
pages = find_core_terms_pages(pdf_file_path, search_term)
print(pages)


21,32,52,68


In [ ]:
!pip install PyMuPDF
import fitz
# 由於無法訪問實際的PDF文件，我們將使用先前用戶上傳的文件進行演示
pdf_file_path = '/content/drive/MyDrive/textmining/Federal District (30).PDF'

# 搜索詞“Core Terms”的變數
search_term = "Core Terms"

# 由於上傳的是圖片，而不是PDF，我們不能直接在這個文件上運行搜索。
# 如果這是一個PDF，代碼將如下運行：
def find_core_terms_pages(pdf_path, search_term):
    doc = fitz.open(pdf_path)
    pages_with_term = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text_instances = page.search_for(search_term)

        if text_instances:
            pages_with_term.append(str(page_num + 1))

    doc.close()
    return ','.join(pages_with_term)

# 這行代碼在實際環境中應該可以執行，但在這裡由於文件類型不匹配會導致錯誤。
pages = find_core_terms_pages(pdf_file_path, search_term)
print(pages)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 37.2 MB/s eta 0:00:00
8,17,21,24,28,31,46,56,59,67,75,83,101,111,118,123,146,151,160,173,186,189,204,211,232,241,258,277,287,303


In [ ]:
# !pip install PyMuPDF
import fitz  # PyMuPDF

def extract_text_with_page_numbers(pdf_path):
    """提取PDF文檔的文本，同時記錄每頁開始的位置"""
    doc = fitz.open(pdf_path)
    full_text = ""
    page_starts = [0]  # 第一頁的開始位置是0

    for page_num in range(len(doc)):
        page_text = doc.load_page(page_num).get_text()
        full_text += page_text
        # 記錄下一頁的開始位置
        page_starts.append(len(full_text))

    return full_text, page_starts

def find_term_pages(full_text, page_starts, term):
    """在提取的文本中搜尋term，並返回包含term的頁碼"""
    pages_with_term = set()
    pos = full_text.find(term)
    while pos != -1:
        # 確定pos在哪一頁
        for i, start in enumerate(page_starts):
            if pos < start:
                pages_with_term.add(i)  # i是頁碼
                break
        pos = full_text.find(term, pos + 1)

    return sorted(pages_with_term)

# 示範用法
pdf_file_path = '/content/drive/MyDrive/textmining/State Surperior Court (CA) (30).PDF'
search_term = "Core Terms"

# 先提取文本和頁面開始位置
full_text, page_starts = extract_text_with_page_numbers(pdf_file_path)

# 再找到包含"Core Terms"的頁碼
pages = find_term_pages(full_text, page_starts, search_term)
print("包含'Core Terms'的頁碼有：", pages)


包含'Core Terms'的頁碼有： [7, 22, 38, 78, 119, 149, 153, 157, 217, 230, 240, 259, 286, 290, 300, 307, 310, 341, 351, 370, 374, 378, 381, 390, 402, 406, 410, 417, 432, 436]
